# RuntimeError: CUDA out of memory. 

关于CUDA GPU显存管理的总结：
- GPU显存占用率和存入的数据尺寸成正相关，越大的数据占用显存越多
- 只要使用了GPU，就至少会占x xx M的显存，且这部分显存无法被释放
- 当一块内存不再被变量所引用时，这块内存就由激活内存转为失活内存，但它仍然存在于这个数据队列中
- 当数据队列达到某个阈值时，CUDA会触发垃圾回收机制，清理失活内存
- 运行torch.cuda.empty_cache()可以手动清理失活内存

那么根据上述理论，就可以得到对应的问题解决方案：
- 调小batch_size  
  本质上是防止GPU数据队列向显存申请的空间大于显存本身

- 检查是否有数据持续存入GPU而未释放

- 训练过程中的测试阶段和验证阶段前插入代码with torch.no_grad()  
  原理是不计算梯度，从而不用GPU加速运算，不会把数据再加到数据队列中

# 消融实验

消融实验（ablation study）是一种科学研究方法，用于确定一个条件或参数对结果的影响程度。当研究者提出了一个新的方案或方法时，消融实验通过逐一控制一个条件或参数，来观察结果的变化，以确定哪个条件或参数对结果的影响更大。

举个例子，假设在目标检测系统中，使用了A、B、C三种方法相加取得了不错的效果。但是我们并不知道这个效果是由于A、B还是C起的作用。为了确定哪个方法对结果的影响更大，我们可以保留A、B，移除C进行实验，观察结果的变化。通过这样的消融实验，我们可以最终确定到底是哪个方法对结果的影响更大。

总结来说，消融实验类似于"控制变量法"，通过逐一控制条件或参数来观察结果的变化，以确定它们对结果的影响程度。

消融实验的优点是可以帮助研究人员理解模型的不同组成部分对整体性能的贡献。通过逐步消除模型中的某些组件或功能，研究人员可以评估这些组件或功能对模型性能的影响。这有助于揭示模型的关键组件和关键功能，以及它们在任务中的作用。

消融实验的局限性在于，它可能无法完全反映真实世界的情况。通过消除某些组件或功能，消融实验可能会导致模型在特定任务上的性能下降，但在实际应用中，这些组件或功能可能是必要的。此外，消融实验可能无法考虑到组件之间的相互作用和复杂性，因此结果可能不完全准确。

总的来说，消融实验是一种有用的方法，可以帮助研究人员理解模型的工作原理和性能贡献，但需要谨慎解释和应用其结果。

既然消融实验的目的是展示各个模块/部分对于系统的作用，那么实验的设计者应当对系统的各个模块的构成以及总体的架构有详细的了解。比如说原始的模型M是由一个模块A组成，而新的设计在M的基础上增加了B和C模块，使得现在的模型成为了A+B+C，那么就要分别单独研究模块B与C对于整个系统的影响，此时实验应该按照下面的表格设计，

| |metric 1|	|...|	metric N|
|---|---|---|---|---|
A (baseline)|||||			
A+B|||||			
A+C	|	||||	
A+B+C (final)|	||||		

在上图中，metric 1 - N 表示N个用来评价系统性能的指标，比如图像分类任务中常用的top1 和 top5 error rate。原始模型M（只含有A模块）会被首先测试，得到的结果会成为baseline用来对比。接下来，分别测试模型A+B与A+C来分别测试B模块与C模块的单独作用。最后，就要把所有的模块都放在一起，也就是模型A+B+C，来测试最终模型的性能。

当模块C是基于模块B时，也就是不存在A+C模型的组合方式时，只需要将上表中对应A+C的行删去即可。

# 梯度消失和梯度爆炸

## 问题描述

目前优化神经网络的方法都是基于BP，即根据损失函数计算的误差通过梯度反向传播的方式，指导深度网络权值的更新优化。其中将误差从末层往前传递的过程需要链式法则（Chain Rule）的帮助，因此反向传播算法可以说是梯度下降在链式法则中的应用。

而链式法则是一个连乘的形式，所以当层数越深的时候，梯度将以指数形式传播。梯度消失问题和梯度爆炸问题一般随着网络层数的增加会变得越来越明显。在根据损失函数计算的误差通过梯度反向传播的方式对深度网络权值进行更新时，得到的梯度值接近0或特别大，也就是梯度消失或爆炸。梯度消失或梯度爆炸在本质原理上其实是一样的。

【梯度消失】经常出现，产生的原因有：一是在深层网络中，二是采用了不合适的损失函数，比如sigmoid。当梯度消失发生时，接近于输出层的隐藏层由于其梯度相对正常，所以权值更新时也就相对正常，但是当越靠近输入层时，由于梯度消失现象，会导致靠近输入层的隐藏层权值更新缓慢或者更新停滞。这就导致在训练时，只等价于后面几层的浅层网络的学习。

【梯度爆炸】一般出现在深层网络和权值初始化值太大的情况下。在深层神经网络或循环神经网络中，误差的梯度可在更新中累积相乘。如果网络层之间的梯度值大于 1.0，那么重复相乘会导致梯度呈指数级增长，梯度变的非常大，然后导致网络权重的大幅更新，并因此使网络变得不稳定。

梯度爆炸会伴随一些细微的信号，如：①模型不稳定，导致更新过程中的损失出现显著变化；②训练过程中，在极端情况下，权重的值变得非常大，以至于溢出，导致模型损失变成 NaN等等。

## 解决方法

梯度消失和梯度爆炸问题都是因为网络太深，网络权值更新不稳定造成的，本质上是因为梯度反向传播中的连乘效应。解决梯度消失、爆炸主要有以下几种方法：

（1） pre-training+fine-tunning

此方法来自Hinton在2006年发表的一篇论文，Hinton为了解决梯度的问题，提出采取无监督逐层训练方法，其基本思想是每次训练一层隐节点，训练时将上一层隐节点的输出作为输入，而本层隐节点的输出作为下一层隐节点的输入，此过程就是逐层“预训练”（pre-training）；在预训练完成后，再对整个网络进行“微调”（fine-tunning）。此思想相当于是先寻找局部最优，然后整合起来寻找全局最优，此方法有一定的好处，但是目前应用的不是很多了。

（2） 梯度剪切：对梯度设定阈值

梯度剪切这个方案主要是针对梯度爆炸提出的，其思想是设置一个梯度剪切阈值，然后更新梯度的时候，如果梯度超过这个阈值，那么就将其强制限制在这个范围之内。这可以防止梯度爆炸。

（3） 权重正则化

另外一种解决梯度爆炸的手段是采用权重正则化（weithts regularization），正则化主要是通过对网络权重做正则来限制过拟合。如果发生梯度爆炸，那么权值就会变的非常大，反过来，通过正则化项来限制权重的大小，也可以在一定程度上防止梯度爆炸的发生。比较常见的是 L1 正则和 L2 正则，在各个深度框架中都有相应的API可以使用正则化。

关于 L1 和 L2 正则化的详细内容可以参考我之前的文章——欠拟合、过拟合及如何防止过拟合

（4） 选择relu等梯度大部分落在常数上的激活函数

relu函数的导数在正数部分是恒等于1的，因此在深层网络中使用relu激活函数就不会导致梯度消失和爆炸的问题。


（5） batch normalization

BN就是通过对每一层的输出规范为均值和方差一致的方法，消除了权重参数放大缩小带来的影响，进而解决梯度消失和爆炸的问题，或者可以理解为BN将输出从饱和区拉倒了非饱和区。


（6） 残差网络的捷径（shortcut）

（7） LSTM的“门（gate）”结构

LSTM全称是长短期记忆网络（long-short term memory networks），LSTM的结构设计可以改善RNN中的梯度消失的问题。主要原因在于LSTM内部复杂的“门”(gates)，LSTM 通过它内部的“门”可以在接下来更新的时候“记住”前几次训练的”残留记忆“。

# 梯度裁剪

梯度裁剪（Gradient Clipping）是一种防止梯度爆炸或梯度消失的优化技术，它可以在反向传播过程中对梯度进行缩放或截断，使其保持在一个合理的范围内。梯度裁剪有两种常见的方法：

按照梯度的绝对值进行裁剪，即如果梯度的绝对值超过了一个阈值，就将其设置为该阈值的符号乘以该阈值。
按照梯度的范数进行裁剪，即如果梯度的范数超过了一个阈值，就将其按比例缩小，使其范数等于该阈值。例如，如果阈值为1，那么梯度的范数就是1。
在PyTorch中，可以使用 `torch.nn.utils.clip_grad_value_` 和 `torch.nn.utils.clip_grad_norm_` 这两个函数来实现梯度裁剪，它们都是在梯度计算完成后，更新权重之前调用的。

在PyTorch中，nn.utils.clip_grad_norm_ 函数用于实现梯度裁剪。这个函数会首先计算出梯度的范数，然后将其限制在一个最大值之内。这样可以防止在反向传播过程中梯度过大导致的数值不稳定问题。

这个函数的参数如下：

- parameters：一个基于变量的迭代器，会进行梯度归一化。通常我们会传入模型的参数，如 model.parameters() 。
- max_norm：梯度的最大范数。如果梯度的范数超过这个值，那么就会对梯度进行缩放，使得其范数等于这个值。
- norm_type：规定范数的类型。默认为2，即L2范数。如果设置为1，则使用L1范数；如果设置为0，则使用无穷范数。

代码的工作流程如下：

- optimizer.zero_grad()：清零所有参数的梯度缓存。
- outputs = model(data)：前向传播，计算模型的输出。
- loss = loss_fn(outputs, target)：计算损失函数。
- loss.backward()：反向传播，计算当前梯度。
- nn.utils.clip_grad_norm_(model.parameters(), max_norm=20, norm_type=2)：对梯度进行裁剪，防止梯度爆炸。
- optimizer.step()：更新模型的参数。

**什么情况下需要梯度裁剪**

梯度裁剪主要用于解决神经网络训练中的梯度爆炸问题。以下是一些可能需要使用梯度裁剪的情况：

（1）深度神经网络：深度神经网络，特别是RNN，在训练过程中容易出现梯度爆炸的问题。这是因为在反向传播过程中，梯度会随着层数的增加而指数级增大。

（2）训练不稳定：如果你在训练过程中观察到模型的损失突然变得非常大或者变为NaN，这可能是梯度爆炸导致的。在这种情况下，使用梯度裁剪可以帮助稳定训练。

（3）长序列训练：在处理长序列数据（如机器翻译或语音识别）时，由于序列长度的增加，梯度可能会在反向传播过程中累加并导致爆炸。梯度裁剪可以防止这种情况发生。

需要注意的是，虽然梯度裁剪可以帮助防止梯度爆炸，但它不能解决梯度消失的问题。对于梯度消失问题，可能需要使用其他技术，如门控循环单元（GRU）或长短期记忆（LSTM）网络，或者使用残差连接等方法。

**注意事项**

梯度裁剪虽然是一种有效防止梯度爆炸的技术，但它也有一些潜在的缺点：

（1）选择合适的裁剪阈值：选择一个合适的梯度裁剪阈值可能会比较困难。如果阈值设置的太大，那么梯度裁剪可能就无法防止梯度爆炸；如果阈值设置的太小，那么可能会限制模型的学习能力。通常，这个阈值需要通过实验来确定。

（2）不能解决梯度消失问题：梯度裁剪只能防止梯度爆炸，但不能解决梯度消失问题。在深度神经网络中，梯度消失也是一个常见的问题，它会导致网络的深层部分难以训练。

（3）可能影响优化器的性能：某些优化器，如Adam和RMSProp，已经包含了防止梯度爆炸的机制。在这些优化器中使用梯度裁剪可能会干扰其内部的工作机制，从而影响训练的效果。

（4）可能引入额外的计算开销：计算和应用梯度裁剪需要额外的计算资源，尤其是在参数量非常大的模型中。